### Import libraries

In [1]:
import numpy as np
import json
import re
import tensorflow as tf
import random
import spacy
nlp = spacy.load('en_core_web_sm')

C:\Users\camper-man\Anaconda3\envs\hemaaaaaaaaaaaaaa\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Import json file

In [2]:
with open('Intent.json') as f:
    intents = json.load(f)

### Preprocessing data
In this part we gonna clean data, split them into inputs and targets tensor, build a tokenizer dictionary and turn sentences into sequences.
The target tensor has a bunch of list with a length of unique title list.

In [3]:
def preprocessing(line):
    line = re.sub(r'[^a-zA-z.?!\']', ' ', line)
    line = re.sub(r'[ ]+', ' ', line)
    return line

In [4]:
# get text and intent title from json data
inputs, targets = [], []
classes = []
intent_doc = {}

for intent in intents['intents']:
    if intent['intent'] not in classes:
        classes.append(intent['intent'])
    if intent['intent'] not in intent_doc:
        intent_doc[intent['intent']] = []
        
    for text in intent['text']:
        inputs.append(preprocessing(text))
        targets.append(intent['intent'])
        
    for response in intent['responses']:
        intent_doc[intent['intent']].append(response)

In [5]:
def tokenize_data(input_list):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='', oov_token='<unk>')
    
    tokenizer.fit_on_texts(input_list)
    
    input_seq = tokenizer.texts_to_sequences(input_list)

    input_seq = tf.keras.preprocessing.sequence.pad_sequences(input_seq, padding='pre')
    
    return tokenizer, input_seq

# preprocess input data
tokenizer, input_tensor = tokenize_data(inputs)

In [6]:
def create_categorical_target(targets):
    word={}
    categorical_target=[]
    counter=0
    for trg in targets:
        if trg not in word:
            word[trg]=counter
            counter+=1
        categorical_target.append(word[trg])
    
    categorical_tensor = tf.keras.utils.to_categorical(categorical_target, num_classes=len(word), dtype='int32')
    return categorical_tensor, dict((v,k) for k, v in word.items())

# preprocess output data
target_tensor, trg_index_word = create_categorical_target(targets)

In [7]:
print('input shape: {} and output shape: {}'.format(input_tensor.shape, target_tensor.shape))

input shape: (143, 9) and output shape: (143, 22)


### Build the model

In [8]:
# hyperparameters
epochs=50
vocab_size=len(tokenizer.word_index) + 1
embed_dim=512
units=128
target_length=target_tensor.shape[1]

In [9]:
# build RNN Model with tensorflow
model = tf.keras.models.Sequential([
    tf.keras.layers.Embedding(vocab_size, embed_dim),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(units, dropout=0.2)),
    tf.keras.layers.Dense(units, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(target_length, activation='softmax')
])

optimizer = tf.keras.optimizers.Adam(lr=1e-2)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 512)         66048     
                                                                 
 bidirectional (Bidirectiona  (None, 256)              656384    
 l)                                                              
                                                                 
 dense (Dense)               (None, 128)               32896     
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 22)                2838      
                                                                 
Total params: 758,166
Trainable params: 758,166
Non-trainable params: 0
__________________________________________________

In [10]:
early_stop = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=4)

# train the model
model.fit(input_tensor, target_tensor, epochs=epochs, callbacks=[early_stop])

Epoch 1/50
5/5 [==============================] - 3s 29ms/step - loss: 3.0902 - accuracy: 0.0490
Epoch 2/50
5/5 [==============================] - 0s 29ms/step - loss: 3.0526 - accuracy: 0.0909
Epoch 3/50
5/5 [==============================] - 0s 29ms/step - loss: 3.0138 - accuracy: 0.1329
Epoch 4/50
5/5 [==============================] - 0s 29ms/step - loss: 2.9502 - accuracy: 0.2028
Epoch 5/50
5/5 [==============================] - 0s 29ms/step - loss: 2.8550 - accuracy: 0.2727
Epoch 6/50
5/5 [==============================] - 0s 28ms/step - loss: 2.7227 - accuracy: 0.3147
Epoch 7/50
5/5 [==============================] - 0s 28ms/step - loss: 2.5323 - accuracy: 0.3357
Epoch 8/50
5/5 [==============================] - 0s 30ms/step - loss: 2.3197 - accuracy: 0.3217
Epoch 9/50
5/5 [==============================] - 0s 29ms/step - loss: 2.0680 - accuracy: 0.4476
Epoch 10/50
5/5 [==============================] - 0s 30ms/step - loss: 1.7919 - accuracy: 0.4895
Epoch 11/50
5/5 [============

In [11]:
def response(sentence):
    sent_seq = []
    doc = nlp(repr(sentence))
    
    # split the input sentences into words
    for token in doc:
        if token.text in tokenizer.word_index:
            sent_seq.append(tokenizer.word_index[token.text])

        # handle the unknown words error
        else:
            sent_seq.append(tokenizer.word_index['<unk>'])

    sent_seq = tf.expand_dims(sent_seq, 0)
    # predict the category of input sentences
    pred = model(sent_seq)

    pred_class = np.argmax(pred.numpy(), axis=1)
    
    # choice a random response for predicted sentence
    return random.choice(intent_doc[trg_index_word[pred_class[0]]]), trg_index_word[pred_class[0]]

# chat with bot
print("Note: Enter 'quit' to break the loop.")
while True:
    input_ = input('You: ')
    if input_.lower() == 'quit':
        break
    res, typ = response(input_)
    print('Bot: {} -- TYPE: {}'.format(res, typ))
    print()

Note: Enter 'quit' to break the loop.
You: how are you?
Bot: That is an difficult question, can you prove that you are? -- TYPE: SelfAware

You: hi
Bot: Hi human  -- TYPE: Greeting

You: how are you
Bot: That is an difficult question, can you prove that you are? -- TYPE: SelfAware

You: ?
Bot: Mr X said he recently read an interview with a man who had several computer chips implanted into him. -- TYPE: Gossip

You: ??
Bot: Jordan said he notice that lots of other people are talking to me how many. -- TYPE: Gossip

You: quit
